In [1]:
import pandas as pd
import requests
import json
import datetime
import tqdm
import time
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count, freeze_support

In [2]:
token = 'aef4a8fcaef4a8fcaef4a8fc0cae9b7455aaef4aef4a8fcf0a5acac7eb6b17369a4d378'

In [3]:
publics = pd.DataFrame({
    'owner_id':['-6050194', '-193186817', '-161610747', '-128411654', '569689032', '-179333837', '-200210651', '-171049283', '-83958801'],
    'domain':['dtpche', 'derevya_cherepovtsa', 'komandacher', 'tos_cher', 'germanovve', 'public_happy_city', 'shagi_che', 'che_104', 'club83958801'],
    'name': ['Деревья череповца', 'Мой Череповец', 'ТОС Череповец', 'публичная страница мэра города', 'Пешеходная зона. Череповец.', 'Пешеходная зона. Череповец.', 'Проектная группа ШАГИ', 'Череповец | Зашекснинский | 104 | Сточик', 'ТОС "Индустриальный”']
})

In [4]:
df_pages = []

In [55]:
def get_wall(owners_id):
    for owner_id in owners_id:

        domain = publics.set_index('owner_id').loc[owner_id, 'domain']
        df = dict_first_page[owner_id]
        df['domain'] = domain
        df_pages.append(df)

        for offset in tqdm.notebook.tqdm(range(100, dict_counts[owner_id], 100)):
            try:
                url = 'https://api.vk.com/method/wall.get?owner_id={}&domain={}&count=100&offset={}&extended=1&access_token={}&v=5.131'.format(owner_id, domain, offset, token)
                page = requests.get(url).text
                response = json.loads(page)['response']
                df = pd.json_normalize(response['items'])
                df['domain'] = domain
                df_pages.append(df)
                time.sleep(1)
                
            except Exception as e:
                print(e)


In [6]:
dict_first_page = {}
dict_counts = {}

In [7]:
for owner_id, domain in publics[['owner_id', 'domain']].values:

    url = 'https://api.vk.com/method/wall.get?owner_id={}&domain={}&count=100&offset={}&extended=1&access_token={}&v=5.131'.format(owner_id, domain, 0, token)
    page = requests.get(url).text
    response = json.loads(page)['response']
    count = int(response['count'])
    dict_counts[owner_id] = count
    dict_first_page[owner_id] = pd.json_normalize(response['items'])

In [58]:
publics['count'] = list(dict_counts.values())

In [60]:
publics.to_csv('data/publics.csv', index = False)

In [56]:
datalist = list(dict_counts.keys())

if __name__== "__main__":
    freeze_support()
    pool = Pool()
    ITERATION_COUNT = cpu_count()-1 if len(datalist) >= cpu_count()-1 else len(datalist)

    count_per_iteration = len(datalist) / float(ITERATION_COUNT)

    for i in range(0, ITERATION_COUNT):
        list_start = int(count_per_iteration * i)
        list_end = int(count_per_iteration * (i+1))
        pool.apply_async(get_wall, [datalist[list_start:list_end]])

In [63]:
df_posts = pd.concat(df_pages, ignore_index = True).drop_duplicates(['date', 'id', 'domain'])

In [64]:
df_posts.shape

(23083, 53)

In [65]:
df_posts['date'] = pd.to_datetime(df_posts.date, unit='s')+datetime.timedelta(hours = 3)

In [66]:
df_posts

,id,from_id,owner_id,date,marked_as_ads,post_type,text,is_pinned,attachments,short_text_rate,...,geo.place.address,geo.place.category,geo.place.city,geo.place.country,geo.place.category_object.id,geo.place.category_object.title,geo.place.category_object.icons,post_source.data,geo.showmap,post_source.url
0,5449,-83958801,-83958801,2022-03-04 11:15:38,0.0,post,"Больше о работе Городской Управы #7, а также о...",1.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5469,-83958801,-83958801,2022-03-20 13:03:33,0.0,post,Всех причастных с праздником!,NaN,NaN,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5468,-83958801,-83958801,2022-03-19 22:26:38,0.0,post,,NaN,NaN,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5467,-83958801,-83958801,2022-03-18 21:27:34,0.0,post,Работа УК,NaN,NaN,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5465,-83958801,-83958801,2022-03-18 07:37:38,0.0,post,,NaN,NaN,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23978,39700,-6050194,-6050194,2018-05-01 11:22:08,0.0,post,,NaN,NaN,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23979,39691,-6050194,-6050194,2018-05-01 10:20:00,0.0,post,По пути на Романду упали столбы\n\n📋[id2170823...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23980,39689,-6050194,-6050194,2018-05-01 10:10:00,0.0,post,Пoд Чеpепoвцoм из-зa пoзднегo сигнaлa o пoжapе...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23981,39682,-6050194,-6050194,2018-05-01 09:41:51,0.0,post,📋[id82798941|Галина Сидорова],NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df_posts.to_csv('data/vk_posts.csv', index = False)

# Проверка

In [70]:
df_posts.groupby('domain').count()[['id']].reset_index().merge(publics[['domain', 'count']], how = 'left', on = 'domain')

,domain,id,count
0,che_104,593,593
1,club83958801,1693,1693
2,derevya_cherepovtsa,1168,1168
3,dtpche,10667,10667
4,germanovve,1581,1581
5,komandacher,2404,2404
6,public_happy_city,298,298
7,shagi_che,130,130
8,tos_cher,4549,4549


# Преобразование данных

In [4]:
df_posts = pd.read_csv('data/vk_posts.csv')

C:\Users\yupes\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (43,48,49,52) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23083 entries, 0 to 23082
Data columns (total 53 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               23083 non-null  int64  
 1   from_id                          23083 non-null  int64  
 2   owner_id                         23083 non-null  int64  
 3   date                             23083 non-null  object 
 4   marked_as_ads                    21502 non-null  float64
 5   post_type                        23083 non-null  object 
 6   text                             17891 non-null  object 
 7   is_pinned                        4 non-null      float64
 8   attachments                      16804 non-null  object 
 9   short_text_rate                  23083 non-null  float64
 10  carousel_offset                  5585 non-null   float64
 11  hash                             23083 non-null  object 
 12  post_source.platfo

In [9]:
df_posts['hash'].drop_duplicates().shape

(23083,)